# Regressão Linear

Neste notebook, vamos explorar a regressão linear em detalhes:
- Visualizações principais
- Dispersão para múltiplas variáveis
- Análise de resíduos
- Predição de preços

In [ ]:
# Importar bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
sns.set(style="whitegrid")

# Configurar exibição de floats
np.set_printoptions(precision=4, suppress=True)

# Carregar dados
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv"
column_names = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"]
data = pd.read_csv(url, header=None, delimiter=", ")
data.columns = column_names

# Visualizar as 5 primeiras linhas
data.head()

## 1. Estatísticas Descritivas

Vamos começar explorando estatísticas descritivas dos dados.

In [ ]:
# Estatísticas descritivas
stats = data.describe().T
stats["IQR"] = stats["75%"] - stats["25%"]  # Intervalo Interquartil
stats["Outliers"] = np.where((data < (stats["25%"] - 1.5 * stats["IQR"])) | (data > (stats["75%"] + 1.5 * stats["IQR"])), "Sim", "Não")

# Exibir estatísticas
stats

## 2. Visualização da Distribuição das Variáveis

Vamos visualizar a distribuição das variáveis numéricas.

In [ ]:
# Histograma e KDE
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(16, 12))
fig.suptitle('Distribuição das Variáveis', fontsize=16, fontweight='bold')

for ax, column in zip(axes.flatten(), data.columns):
    sns.histplot(data[column], kde=True, ax=ax, color='skyblue', bins=30)
    ax.set_title(column, fontsize=12, fontweight='bold')
    ax.set_xlabel('')
    ax.set_ylabel('')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## 3. Correlações entre Variáveis

Analisando correlações para entender relações entre variáveis.

In [ ]:
# Matriz de correlação
correlation_matrix = data.corr()

# Heatmap da matriz de correlação
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5)

# Título
plt.title('Mapa de Calor da Correlação entre Variáveis', fontsize=16, fontweight='bold')

# Exibir
plt.show()

## 4. Preparação dos Dados para Regressão

Nesta etapa, vamos preparar os dados para o modelo de regressão linear.

In [ ]:
# Dividir em variáveis explicativas (X) e variável alvo (y)
X = data.drop("MEDV", axis=1)
y = data["MEDV"]

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Criar o modelo de regressão linear
model = LinearRegression()

# Treinar o modelo
model.fit(X_train_scaled, y_train)

# Fazer predições
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

# Avaliar o modelo
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

# Resultados
print(f"Conjunto de Treino: MSE = {mse_train:.4f}, R² = {r2_train:.4f}")
print(f"Conjunto de Teste: MSE = {mse_test:.4f}, R² = {r2_test:.4f}")

## 5. Visualizações das Predições

Vamos visualizar as predições do modelo.

In [ ]:
# Gráfico: Valores Reais vs Preditos
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, alpha=0.7, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valores Reais')
plt.ylabel('Valores Preditos')
plt.title('Valores Reais vs Preditos')
plt.grid(True)
plt.show()

## 6. Visualizações Principais

Vamos criar visualizações para entender melhor o modelo:
1. Valores Reais vs Preditos
2. Dispersão para Múltiplas Variáveis (Top 4 Features)
3. Importância das Features

In [ ]:
# Calcular predições
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

# Criar figura com subplots
fig = plt.figure(figsize=(20, 6))

# Gráfico 1: Valores Reais vs Preditos
ax1 = plt.subplot(1, 3, 1)
ax1.scatter(y_test, y_pred_test, alpha=0.5, edgecolors='k', linewidth=0.5)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax1.set_xlabel('Valores Reais', fontsize=12, fontweight='bold')
ax1.set_ylabel('Valores Preditos', fontsize=12, fontweight='bold')
ax1.set_title('Valores Reais vs Preditos\n(Linha vermelha = predição perfeita)', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Gráfico 2: Dispersão da Feature Mais Importante
ax2 = plt.subplot(1, 3, 2)
feature_importance = np.abs(model.coef_)
most_important_idx = np.argmax(feature_importance)
most_important_feature = X.columns[most_important_idx]

ax2.scatter(X_test_scaled[:, most_important_idx], y_test, alpha=0.5, label='Real', color='blue', s=30)
ax2.scatter(X_test_scaled[:, most_important_idx], y_pred_test, alpha=0.5, label='Predito', color='red', s=30)
ax2.set_xlabel(most_important_feature, fontsize=12, fontweight='bold')
ax2.set_ylabel('Preço (MEDV)', fontsize=12, fontweight='bold')
ax2.set_title(f'Feature Mais Importante: {most_important_feature}\nCoeficiente: {model.coef_[most_important_idx]:.4f}', 
              fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gráfico 3: Importância das Features (Top 10)
ax3 = plt.subplot(1, 3, 3)
top_n = 10
top_indices = np.argsort(feature_importance)[-top_n:][::-1]
top_features = [X.columns[i] for i in top_indices]
top_weights = feature_importance[top_indices]

colors = plt.cm.viridis(np.linspace(0, 1, top_n))
bars = ax3.barh(range(top_n), top_weights, color=colors, edgecolor='black')
ax3.set_yticks(range(top_n))
ax3.set_yticklabels(top_features)
ax3.set_xlabel('Importância (|Coeficiente|)', fontsize=12, fontweight='bold')
ax3.set_title(f'Top {top_n} Features Mais Importantes', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='x')
ax3.invert_yaxis()

plt.tight_layout()
plt.show()

## 7. Dispersão para Múltiplas Variáveis

Análise detalhada das 4 features mais importantes.

In [ ]:
# Selecionar as 4 features mais importantes
feature_importance = np.abs(model.coef_)
top_features_idx = np.argsort(feature_importance)[-4:][::-1]
top_features = [X.columns[i] for i in top_features_idx]

# Criar subplot para cada feature
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análise de Dispersão: Features Principais vs Preço', fontsize=16, fontweight='bold')

for idx, (ax, feature_idx, feature_name) in enumerate(zip(axes.flat, top_features_idx, top_features)):
    # Dados de treino e teste (valores originais, não escalados)
    x_train_feat = X_train.iloc[:, feature_idx].values
    x_test_feat = X_test.iloc[:, feature_idx].values
    
    # Scatter plot
    ax.scatter(x_train_feat, y_train, alpha=0.3, s=20, label='Treino', color='blue', edgecolors='none')
    ax.scatter(x_test_feat, y_test, alpha=0.3, s=20, label='Teste', color='green', edgecolors='none')
    
    # Linha de tendência
    z = np.polyfit(x_train_feat, y_train, 1)
    p = np.poly1d(z)
    x_line = np.linspace(x_train_feat.min(), x_train_feat.max(), 100)
    ax.plot(x_line, p(x_line), "r--", linewidth=2, label='Tendência Linear')
    
    # Calcular correlação
    correlation = np.corrcoef(x_train_feat, y_train)[0, 1]
    
    ax.set_xlabel(feature_name, fontsize=11, fontweight='bold')
    ax.set_ylabel('Preço (MEDV)', fontsize=11, fontweight='bold')
    ax.set_title(f'{feature_name}\nCorrelação: {correlation:.3f} | Coef: {model.coef_[feature_idx]:.3f}', 
                 fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Interpretação das Features Principais:")
for i, (feature_idx, feature_name) in enumerate(zip(top_features_idx, top_features)):
    coef = model.coef_[feature_idx]
    correlation = np.corrcoef(X_train.iloc[:, feature_idx].values, y_train)[0, 1]
    print(f"{i+1}. {feature_name}:")
    print(f"   Coeficiente: {coef:.4f} | Correlação: {correlation:.4f}")
    print(f"   {'Impacto positivo' if coef > 0 else 'Impacto negativo'} no preço\n")

## 8. 🎁 BÔNUS: Análise de Resíduos

A análise de resíduos é fundamental para validar as premissas da regressão linear:
1. **Linearidade**: Relação linear entre X e y
2. **Homocedasticidade**: Variância constante dos erros
3. **Normalidade**: Resíduos seguem distribuição normal
4. **Independência**: Ausência de padrões nos resíduos

### O que são Resíduos?
Resíduos são as diferenças entre os valores reais e preditos: $e_i = y_i - \hat{y}_i$

In [ ]:
from scipy import stats

# Calcular resíduos
residuals = y_test - y_pred_test

# Criar figura com 4 subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('🎁 ANÁLISE DE RESÍDUOS - Validação das Premissas da Regressão Linear', 
             fontsize=16, fontweight='bold', y=1.00)

# 1. Resíduos vs Valores Preditos
axes[0, 0].scatter(y_pred_test, residuals, alpha=0.5, edgecolors='k', linewidth=0.5, s=30)
axes[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2, label='Resíduo = 0')
axes[0, 0].set_xlabel('Valores Preditos', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Resíduos', fontsize=12, fontweight='bold')
axes[0, 0].set_title('1. Resíduos vs Predições\n✓ Padrão aleatório indica homocedasticidade', 
                     fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Histograma dos Resíduos com Curva Normal
axes[0, 1].hist(residuals, bins=50, density=True, alpha=0.7, color='skyblue', edgecolor='black', label='Resíduos')
axes[0, 1].axvline(x=0, color='r', linestyle='--', linewidth=2, label='Média = 0')

# Sobrepor curva normal
mu, sigma = residuals.mean(), residuals.std()
x = np.linspace(residuals.min(), residuals.max(), 100)
axes[0, 1].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label=f'Normal(μ={mu:.2f}, σ={sigma:.2f})')

axes[0, 1].set_xlabel('Resíduos', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Densidade', fontsize=12, fontweight='bold')
axes[0, 1].set_title('2. Distribuição dos Resíduos\n✓ Deve seguir distribuição normal', 
                     fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Q-Q Plot (Quantil-Quantil)
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].get_lines()[0].set_markerfacecolor('blue')
axes[1, 0].get_lines()[0].set_markeredgecolor('black')
axes[1, 0].get_lines()[0].set_markersize(5)
axes[1, 0].get_lines()[1].set_color('red')
axes[1, 0].get_lines()[1].set_linewidth(2)
axes[1, 0].set_title('3. Q-Q Plot\n✓ Pontos na linha diagonal indicam normalidade', 
                     fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Resíduos Padronizados (Scale-Location Plot)
standardized_residuals = (residuals - np.mean(residuals)) / np.std(residuals)
axes[1, 1].scatter(y_pred_test, np.abs(standardized_residuals), alpha=0.5, edgecolors='k', linewidth=0.5, s=30)
axes[1, 1].axhline(y=0, color='gray', linestyle='-', linewidth=1)
axes[1, 1].axhline(y=2, color='orange', linestyle=':', linewidth=2, label='±2σ (95%)')
axes[1, 1].axhline(y=3, color='red', linestyle=':', linewidth=2, label='±3σ (99.7%)')

# Adicionar linha de tendência
from scipy.interpolate import make_interp_spline
x_smooth = np.linspace(y_pred_test.min(), y_pred_test.max(), 100)
indices = np.argsort(y_pred_test)
try:
    spl = make_interp_spline(y_pred_test[indices][::10], np.abs(standardized_residuals)[indices][::10], k=3)
    y_smooth = spl(x_smooth)
    axes[1, 1].plot(x_smooth, y_smooth, 'b-', linewidth=2, alpha=0.5, label='Tendência')
except:
    pass

axes[1, 1].set_xlabel('Valores Preditos', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('|Resíduos Padronizados|', fontsize=12, fontweight='bold')
axes[1, 1].set_title('4. Scale-Location Plot\n✓ Linha horizontal indica variância constante', 
                     fontsize=12, fontweight='bold')
axes[1, 1].legend(loc='upper right')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas dos resíduos
print("\n" + "="*70)
print("📊 ESTATÍSTICAS DOS RESÍDUOS")
print("="*70)
print(f"{'Métrica':<30} {'Valor':>15} {'Interpretação'}")
print("-"*70)
print(f"{'Média':<30} {np.mean(residuals):>15.6f} {'✓ Próximo de 0' if abs(np.mean(residuals)) < 0.01 else '✗ Distante de 0'}")
print(f"{'Desvio Padrão':<30} {np.std(residuals):>15.4f}")
print(f"{'Mínimo':<30} {np.min(residuals):>15.4f}")
print(f"{'Máximo':<30} {np.max(residuals):>15.4f}")
print(f"{'Mediana':<30} {np.median(residuals):>15.4f} {'✓ Próximo de 0' if abs(np.median(residuals)) < 0.01 else '✗ Distante de 0'}")

# Teste de normalidade (Shapiro-Wilk)
sample_size = min(5000, len(residuals))
sample_residuals = np.random.choice(residuals, sample_size, replace=False)
statistic, p_value = stats.shapiro(sample_residuals)

print("\n" + "="*70)
print("🔬 TESTE DE NORMALIDADE (Shapiro-Wilk)")
print("="*70)
print(f"Estatística W: {statistic:.6f}")
print(f"p-valor: {p_value:.6f}")
print(f"Conclusão (α=0.05): {'✓ Resíduos são normais' if p_value > 0.05 else '✗ Resíduos não são perfeitamente normais'}")
print(f"\nInterpretação: {'Os resíduos seguem distribuição normal' if p_value > 0.05 else 'Há desvios da normalidade'}")

# Contar outliers
outliers_2sigma = np.sum(np.abs(standardized_residuals) > 2)
outliers_3sigma = np.sum(np.abs(standardized_residuals) > 3)
total = len(residuals)

print("\n" + "="*70)
print("⚠️ DETECÇÃO DE OUTLIERS")
print("="*70)
print(f"Total de observações: {total}")
print(f"Além de ±2σ (esperado ~5%): {outliers_2sigma} ({outliers_2sigma/total*100:.2f}%)")
print(f"Além de ±3σ (esperado ~0.3%): {outliers_3sigma} ({outliers_3sigma/total*100:.2f}%)")

# Avaliação geral
print("\n" + "="*70)
print("✅ AVALIAÇÃO GERAL DAS PREMISSAS")
print("="*70)

checks = []
checks.append(("Média dos resíduos ≈ 0", abs(np.mean(residuals)) < 0.1))
checks.append(("Normalidade (Shapiro-Wilk)", p_value > 0.05))
checks.append(("Outliers dentro do esperado", outliers_2sigma/total < 0.10))

for check, passed in checks:
    status = "✓" if passed else "✗"
    print(f"{status} {check}")

passed_checks = sum([c[1] for c in checks])
print(f"\n{'='*70}")
print(f"Resultado: {passed_checks}/{len(checks)} premissas atendidas")
if passed_checks == len(checks):
    print("🎉 Modelo atende bem às premissas da regressão linear!")
elif passed_checks >= len(checks) * 0.6:
    print("⚠️ Modelo atende parcialmente às premissas. Considere transformações nos dados.")
else:
    print("❌ Modelo não atende às premissas. Considere outros métodos de regressão.")
print("="*70)